In [1]:
# Import libraries
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Movies with genres, movie ID and year
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bf63d8034a7a4cfe81617b85e5075c04 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='8vf1KdAjYDuS5Josd_IH_vmehb7EfGU7eRBglM1Zgwlq',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_bf63d8034a7a4cfe81617b85e5075c04.get_object(Bucket='movierecommender-donotdelete-pr-8pww64483vjeew',Key='movies.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
movies_df = df_data_1
movies_df.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies_df.shape

(34208, 3)

In [4]:
# Create new column for year
movies_df['year'] = movies_df['title']
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji (1995)
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II (1995)


In [5]:
# Get rid of year from title column
movies_df['title'] = movies_df['title'].replace(regex=r'\(\d\d\d\d\)', value='')
# Get rid of any trailing white spaces in title
movies_df['title'] = movies_df['title'].str.strip()
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)
1,2,Jumanji,Adventure|Children|Fantasy,Jumanji (1995)
2,3,Grumpier Old Men,Comedy|Romance,Grumpier Old Men (1995)
3,4,Waiting to Exhale,Comedy|Drama|Romance,Waiting to Exhale (1995)
4,5,Father of the Bride Part II,Comedy,Father of the Bride Part II (1995)


In [6]:
movies_df['year'] = movies_df['year'].replace(regex=r'.*\(', value='')
movies_df['year'] = movies_df['year'].replace(regex=r'\)$', value='')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [7]:
# User Ratings
body = client_bf63d8034a7a4cfe81617b85e5075c04.get_object(Bucket='movierecommender-donotdelete-pr-8pww64483vjeew',Key='userRatings.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
user_ratings_df = df_data_2
user_ratings_df.head()

,Title,Rating
0,"Matrix, The",5.0
1,Risen,5.0
2,Toy Story,4.5
3,"Breakfast Club, The",4.5
4,Jumanji,1.0


In [8]:
# Ratings of movies
body = client_bf63d8034a7a4cfe81617b85e5075c04.get_object(Bucket='movierecommender-donotdelete-pr-8pww64483vjeew',Key='ratings.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
ratings_df = df_data_3
ratings_df.head()


,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [30]:
# Drop timestamp
ratings_df = ratings_df.drop(columns=['timestamp'])
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [9]:
# The ratings_df has ratings of movies with movieId. It doesn't have movie title. So in the user_ratings_df, we want
# to add movieId to the movies that the user rated
# First, let's rename title to Title in movies_df, so that we can use join on the movies_df and user_ratings_df 
# dataframes
movies_df.rename(columns={'title': 'Title'}, inplace=True)
movies_df.head()


,movieId,Title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [10]:
# Then let's merge movies_df and user_ratings_df to get a dataframe with movieID,
# Title, Rating
user_ratings_df = pd.merge(movies_df, user_ratings_df, left_on='Title', right_on='Title')
user_ratings_df.head()

,movieId,Title,genres,year,Rating
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,4.5
1,2,Jumanji,Adventure|Children|Fantasy,1995,1.0
2,296,Pulp Fiction,Comedy|Crime|Drama|Thriller,1994,2.5
3,377,Speed,Action|Romance|Thriller,1994,5.0
4,1274,Akira,Action|Adventure|Animation|Sci-Fi,1988,0.0


In [11]:
# Get rid of genres and year columns
user_ratings_df.drop(axis=1, columns=['genres', 'year'], inplace=True)
user_ratings_df

,movieId,Title,Rating
0,1,Toy Story,4.5
1,2,Jumanji,1.0
2,296,Pulp Fiction,2.5
3,377,Speed,5.0
4,1274,Akira,0.0
5,1407,Scream,5.0
6,1968,"Breakfast Club, The",4.5
7,2571,"Matrix, The",5.0
8,151687,Risen,5.0
9,151701,Bloodmoney,2.0


In [12]:
# Find which users have watched and rated the movies that our user has watched
user_subset_df = ratings_df[ratings_df['movieId'].isin(user_ratings_df['movieId'])]
user_subset_df.drop(axis=1, columns=['timestamp'], inplace=True)
user_subset_df.reset_index(drop=True, inplace=True)
user_subset_df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,userId,movieId,rating
0,2,2571,3.5
1,4,296,4.0
2,4,377,4.0
3,4,1407,4.0
4,4,2571,4.0


In [21]:
# Sort users that have the most movies in common with our user. These users will get highest priority.
user_subset_group = user_subset_df.groupby(['userId'])
user_subset_group = sorted(user_subset_group,  key=lambda x: len(x[1]), reverse=True)

In [22]:
# We want to find Pearson Correlation between users from user_subset_group and our user. But it'll be time
# consuming to do this for every single user. So we just take the top 100 users.
user_subset_group = user_subset_group[0:100]

In [23]:
# Find Pearson correlation between users

pearson_correlation = {}

for name, group in user_subset_group:
    n_value = len(group)
    temp_df = user_ratings_df[user_ratings_df['movieId'].isin(group['movieId'])]
    temp_rating_list = temp_df['Rating'].to_list()
    temp_group_list = group['rating'].tolist()
    Sxx = sum([i**2 for i in temp_rating_list]) - pow(sum(temp_rating_list),2)/float(n_value)
    Syy = sum([i**2 for i in temp_group_list]) - pow(sum(temp_group_list),2)/float(n_value)
    Sxy = sum( i*j for i, j in zip(temp_rating_list, temp_group_list)) - sum(temp_rating_list)*sum(temp_group_list)/float(n_value)

    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearson_correlation[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearson_correlation[name] = 0
       

In [33]:
pearson_df = pd.DataFrame.from_dict(pearson_correlation, orient='index')
pearson_df.columns = ['similarityIndex']
pearson_df['userId'] = pearson_df.index
pearson_df.index = range(len(pearson_df))
pearson_df.shape

(100, 2)

In [34]:
top_users=pearson_df.sort_values(by='similarityIndex', ascending=False)[0:50]
top_users.head()

,similarityIndex,userId
51,0.872929,14588
17,0.791216,4208
83,0.790250,22950
16,0.754299,4099
62,0.747087,17735


In [36]:
top_users_rating = top_users.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
top_users_rating.head()

,similarityIndex,userId,movieId,rating
0,0.872929,14588,1,4.0
1,0.872929,14588,2,3.5
2,0.872929,14588,3,3.0
3,0.872929,14588,5,2.0
4,0.872929,14588,6,4.0


In [37]:
# Multiply the similarity by the user's ratings
top_users_rating['weightedRating'] = top_users_rating['similarityIndex']*top_users_rating['rating']
top_users_rating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.872929,14588,1,4.0,3.491717
1,0.872929,14588,2,3.5,3.055252
2,0.872929,14588,3,3.0,2.618788
3,0.872929,14588,5,2.0,1.745858
4,0.872929,14588,6,4.0,3.491717


In [39]:
# Apply a sum to the top_users after grouping it up by userId
temp_top_users_rating = top_users_rating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
temp_top_users_rating.columns = ['sum_similarityIndex','sum_weightedRating']
temp_top_users_rating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,19.464124,77.741787
2,19.464124,52.773637
3,8.827677,24.266164
4,1.763827,4.580887
5,8.470583,20.851462


In [40]:
# Create an empty dataframe
recommendation_df = pd.DataFrame()
# Now we take the weighted average
recommendation_df['weighted average recommendation score'] = temp_top_users_rating['sum_weightedRating']/temp_top_users_rating['sum_similarityIndex']
recommendation_df['movieId'] = temp_top_users_rating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.994107,1
2,2.711329,2
3,2.748873,3
4,2.597129,4
5,2.461632,5


In [41]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
4403,5.0,4403
1046,5.0,1046
103210,5.0,103210
2647,5.0,2647
2634,5.0,2634
892,5.0,892
2607,5.0,2607
1056,5.0,1056
55067,5.0,55067


In [42]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,Title,genres,year
875,892,Twelfth Night,Comedy|Drama|Romance,1996
1027,1046,Beautiful Thing,Drama|Romance,1996
1036,1056,Jude,Drama,1996
2523,2607,Get Real,Drama|Romance,1998
2550,2634,"Mummy, The",Horror,1959
2562,2647,House of Frankenstein,Horror,1944
4309,4403,"Fall of the House of Usher, The (House of Usher)",Horror,1960
12110,55063,My Winnipeg,Documentary|Fantasy,2007
12111,55067,Requiem,Drama|Thriller,2006
21258,103210,Fullmetal Alchemist: The Sacred Star of Milos,Action|Adventure|Animation,2011
